# RNNを用いた言語モデル

RNNを用いて言語モデルを実装する。

前章では、深層学習を活用してある単語から次の単語を予測するモデルを作成した。しかし、これでは文脈全体を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。

RNNを用いることで、文脈を考慮した予測が可能になる。  
本章ではRNNの構造について学び、RNNを用いた言語モデルを実装する。

In [1]:
import math
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 学習データの用意

In [3]:
n_data = 20000

wiki40b

In [4]:
text_path = "data/jawiki.txt"
with open(text_path) as f:
    data = f.read().splitlines()

In [5]:
data = data[:n_data]
text_path = f"data/jawiki_{n_data}.txt"
with open(text_path, 'w') as f:
    f.write("\n".join(data))

print("num of data:", len(data))
data[:3] # examples

num of data: 20000


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']


---

## RNN

*Recurrent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

RNNは、時系列データにおけるある時間$t$のデータ$x_t$に対して以下のような演算を行い、出力$h_t$を決定する。

$$
\begin{align}
h_t
    &= \text{tanh}(\text{fc}_x(x_t) + \text{fc}_h(h_{t-1})) \\
    &= \text{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h) \\
\end{align}
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。$\text{fc}$は全結合層。

演算内容はシンプルで、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数（tanh）に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。活性化関数はtanhでなくてもよいが、tanhが使われることが多い。

ちなみに、このRNNからの出力$h_t$は**隠れ状態**と呼ぶ。

この単一時間の演算を行うRNNを`RNNCell`として実装してみる。

In [6]:
class RNNCell(nn.Module):
    def __init__(self, input_size: int, output_size: int):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        """
            x: (batch_size, input_size)
            h: (batch_size, output_size)
        """
        z = self.fc_input(x) + self.fc_output(h) # (batch_size, output_size)
        h = F.tanh(z)
        return h

推論の流れを見てみる。適当に出力の初期値$h_0$と初めの入力$x_1$を定義する。$h_0$は0ベクトルで良い。

In [7]:
batch_size, input_size, hidden_size = 2, 3, 4
x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)

これをRNNに入力することで時間$t=1$の出力$h_1$を得る。

In [8]:
rnn = RNNCell(input_size, hidden_size)
h1 = rnn(x1, h0)
h1

tensor([[ 0.0312,  0.1894, -0.2690,  0.4758],
        [ 0.3340,  0.3699,  0.1621,  0.2997]], grad_fn=<TanhBackward0>)

そしてこの$h_1$を参照して次の時間の入力$x_2$に対する出力$h_2$を決定する。

In [9]:
x2 = torch.randn(batch_size, input_size)
h2 = rnn(x2, h1)
h2

tensor([[ 0.6070, -0.6822,  0.2284,  0.7123],
        [-0.4832,  0.7584, -0.5546,  0.8058]], grad_fn=<TanhBackward0>)

これがRNNの推論の流れである。この一連の流れをまとめ、全ての時間の演算を一度に行うRNNを`RNN`として実装する。

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h):
        """
            x: (seq_len, batch_size, input_size)
            h: (batch_size, hidden_size)
        """
        hs = []
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        return hs

任意の長さの入力に対して、同じ数の隠れ状態を出力する。

In [11]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(seq_len, batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([5, 2, 4])

少し扱いやすくしたいので書き換える。以下を実現する。
- 系列長の次元ではなくバッチ次元を初めにした形状`(batch_size, seq_len, hidden_size)`で入出力を扱えるようにする。
- 隠れ状態`h`の入力がなかった場合に自動で0ベクトルになるようにする。

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, h=None):
        """
            x: (batch_size, seq_len, input_size)
            h: (batch_size, hidden_size)
        """
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        hs = []
        x = x.transpose(0, 1) # (seq_len, batch_size, input_size)
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        hs = hs.transpose(0, 1) # (batch_size, seq_len, hidden_size)
        return hs

`transpose(0, 1)`でバッチ次元と系列長の次元を入れ替える事で実現した。

In [13]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(batch_size, seq_len, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([2, 5, 4])

### PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

#### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。初めに実装したもの。  
https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html

In [14]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn_cell = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, "\n")

Parameter containing:
tensor([[-0.3677,  0.3839, -0.1802],
        [ 0.2928,  0.0675, -0.4474],
        [ 0.3797,  0.1333, -0.2918],
        [-0.0545, -0.5167,  0.3064]], requires_grad=True) 

Parameter containing:
tensor([ 0.1016, -0.3543, -0.2831,  0.1527], requires_grad=True) 

Parameter containing:
tensor([[-0.2299,  0.4509,  0.3698,  0.4770],
        [-0.2641, -0.4745,  0.4616, -0.4993],
        [ 0.0369,  0.4434, -0.4626,  0.1914],
        [ 0.0938, -0.1390, -0.2475,  0.1540]], requires_grad=True) 

Parameter containing:
tensor([-0.3863, -0.2226, -0.1509, -0.0930], requires_grad=True) 



ちゃんとパラメータが4つあるね。

推論も同じ。

In [15]:
x = torch.randn(batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)
rnn_cell(x, h)

tensor([[-0.0821,  0.9306, -0.3934,  0.3623],
        [ 0.9154,  0.5213, -0.7524,  0.3188]], grad_fn=<TanhBackward0>)

ちなみに$h$は入力しなくてもいい。その場合は勝手に0ベクトルになる。

In [16]:
rnn_cell(x)

tensor([[-0.0821,  0.9306, -0.3934,  0.3623],
        [ 0.9154,  0.5213, -0.7524,  0.3188]], grad_fn=<TanhBackward0>)

#### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [17]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_len, input_size)
hs, h = rnn(x)
hs.shape

torch.Size([2, 5, 4])

全ての時間の隠れ状態が出力される。`batch_first=True`とすることでバッチ次元を初めできる。

最後の時間の隠れ状態も別で出力される。

In [18]:
hs[:, -1] == h

tensor([[[True, True, True, True],
         [True, True, True, True]]])

In [19]:
h.shape

torch.Size([1, 2, 4])

最初の次元はレイヤー数。このクラスでは`num_layers`によってRNNを複数重ねられるようになっている。デフォルトが1。


---

## RNNを用いた言語モデル

RNNを用いた言語モデル構築の流れを見てみる。

基本的に、RNNそのものを1つのモデルとして扱うことはない。RNNはNNの中の1つの層として扱う。なお、RNNが組み込まれたNNをRNNと呼ぶこともある。混乱を避けるために、本章では層としてのRNNはRNN層と呼ぶことにする。次章以降では基本分けないので察して読んで。

ある時間$t$におけるRNN層は、その時間の入力$x_t$と前の時間のRNN層の出力$h_{t-1}$を受け取り、$h_t$を出力する。そしてその$h_t$は次の層（線形層など）に渡される。

では、RNN層をNNに取り入れて言語モデルを作ろう。以下のような最低限のネットワークを構築する。

In [20]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs = self.rnn(x, h) # hs: (batch_size, seq_length, hidden_size)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        h = hs[:, -1] # (batch_size, hidden_size)
        return y, h

このモデルで再帰的な推論を行うことで文章生成が可能になる。初めの単語$w_0$（`bos`）と隠れ状態$h_0$（0ベクトル）を入力し、次の単語$w_1$と隠れ状態$h_1$を得る。次に$w_1$と$h_1$を再度モデルに入力し、$w_2$と$h_2$を得る。これを繰り返すことで、文脈を考慮した文章生成を実現する。

流れを見てみよう。

In [21]:
n_vocab, embed_dim, hidden_size = 8000, 512, 512
model = LanguageModel(n_vocab, embed_dim, hidden_size)

次に初めの単語$w_0$を用意する。

In [22]:
w0 = torch.tensor(0).reshape(1, -1) # (batch_size, seq_length) = (1, 1)
w0

tensor([[0]])

初めの隠れ状態$h_0$も用意し、これらをモデルに入力する。

In [23]:
h0 = torch.zeros(1, hidden_size)
y1, h1 = model(w0, h0)
y1.shape, h1.shape

(torch.Size([1, 1, 8000]), torch.Size([1, 512]))

出力を確率分布に変換し、次の単語をサンプリングする。

In [24]:
dist = F.softmax(y1.squeeze(), dim=-1) # 確率分布
w1, = random.choices(range(n_vocab), weights=dist)
w1

4928

これを繰り返すことで文章を生成する。

ある時間$t$で参照するのは、その時間の入力$x_t$と前の時間の隠れ状態$h_{t-1}$のみである。推論時は$x_t$が前の時間の出力$y_{t-1}$になる。なのになぜ（$t-2$以前の情報を含めた）文脈全体を考慮できるのだろう。やっていることは前章にNNで作ったマルコフモデルと同じように見えるのに。

これは、RNNが参照している$h_t$に$t$より前の情報も含まれるからである。前章で作ったNNはある時間$t$の演算で前の時間の出力$y_{t-1}$のみを参照していることになる。しかし$y_{t-1}$はある一つの単語を表すクラスラベルであり、そこには当然単語一つ分の情報しか乗らない。一方で隠れ状態$h_{t-1}$は連続的なベクトルであるため、多くの情報を乗せることができる。実際に学習を進めることで、$h_{t}$には$y_t$の適切な予測に必要な情報が乗るようになる。


---

## 学習データの作成

モデルの学習に使用する、入力と出力のペアを作成する。どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。

RNNは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を3つの時間の入力$x_1,x_2,x_3$として与えたとき、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に予測する。そしてこの3つの単語の誤差とその勾配は一度に求められる（詳細は次のBPTTの節にて）。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。入力サンプル$x_1^{(n)},x_2^{(n)},\cdots,x_3^{(n)}$とそれに対応する正解$y_1^{(n)},y_2^{(n)},\cdots,y_3^{(n)}$。こうすれば文脈と正解の組み合わせが全て網羅できる。なお本当はBOSやEOSが入る。

ではこれに合わせて`Dataset`を作成する。

In [25]:
class TextDataset(Dataset):
    def __init__(self, data_ids: List[List[int]]):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        in_text = text[:-1] # 入力単語列
        out_text = text[1:] # 入力を1つずつずらした単語列
        return in_text, out_text

    def __len__(self):
        return self.n_data


---

## BPTT

*Back Propagation Through Time*

時間を跨いだ逆伝播。

先でも述べたが、RNNは、ある時間$t$における演算でその時間の入力$x_t$以外に隠れ状態（前のRNN層からの出力）$h_{t-1}$を参照する。これは$x_t$のみを参照する通常のNNとは大きく異なる点である。

言語モデルの様な、各時間で確率分布からのサンプリングを挟むようなモデルでは、この隠れ状態が重要な役割を果たす。それは**時間を跨いで逆伝播を繋げること**である。この時間を跨いだ逆伝播をBPTT (*Back Propagation Through Time*) と呼ぶ。前章で作成したNNのように、$h_{t-1}$を用いず、$x_t$（$y_{t-1}$）のみを演算に用いる場合、時間を跨いだ逆伝播はできない。$y_t$が確率分布からのサンプリングを経て得られるためである。これは微分できない。

BPTTによって時間を遡って勾配を届けられるようになり、長期的な視点で誤差が減るように学習できる。その際、勾配の伝達を担う隠れ状態には、長期的な文脈の情報が載るようになることが期待される。

ちなみに、確率分布からのサンプリングを挟まない場合は普通に逆伝播が届く。モデルの出力値をそのまま入力する場合とか。あとは、サンプリングといっても、出力されたデータの一部を次に入力するような場合も普通に逆伝播が届く。1と0からなるベクトルを掛けているように解釈ができ、この演算は微分ができるから。

### Truncated BPTT

BPTTで細かく時間を区切る手法。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。

隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなる。ただ、実はRNNの時点で長期的な（より長い時間での）文脈の考慮は難しいため、大きな影響にはならない。

実装を見ると分かり易いかも。こんな感じになる。

```python
trunc_len = 64 # 区切る長さ

for x, t in dataloader:
    h = None # 隠れ状態の初期化

    # Truncated BPTT
    for i in range(0, x.shape[1], trunc_len):
        x_batch = x[:, i:i+trunc_len].to(device)
        t_batch = t[:, i:i+trunc_len].to(device)

        optimizer.zero_grad()
        y, h = model(x_batch, h)
        loss = loss_fn(y, t_batch)
        loss.backward()
        optimizer.step()

        h = h.detach() # 計算グラフから切り離す
```

あくまでも区切るのは逆伝播だけなので、順伝播時の隠れ状態はちゃんと保持される。


---

## ミニバッチ学習

RNNを用いた言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [26]:
from torch.nn.utils.rnn import pad_sequence

In [27]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [ ]:
pad_id = 3
vocab_size = 8000
tokenizer_prefix = f"models/tokenizer_jawiki_{n_data}"

spm.SentencePieceTrainer.Train(
    input=text_path,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

これでトークン化。

In [29]:
sp = spm.SentencePieceProcessor(f"{tokenizer_prefix}.model")
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

n_vocab = len(sp)
print("num of vocabrary:", n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 12, 20, 528, 495, 283, 48, 558, 52, 3542]

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [30]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 64
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample_x, sample_y = next(iter(train_loader))
print("input batch shape", sample_x.shape)
sample_x[0][:10], sample_y[0][:10] # example

input batch shape torch.Size([64, 1951])


(tensor([   1,   12,  439,  208,   90, 4249,  103,    7, 1280, 2572]),
 tensor([  12,  439,  208,   90, 4249,  103,    7, 1280, 2572,  208]))

これで学習データはOK。

最後に、学習時に使う交差エントロピーでパディング用のトークンを無視するように定義すれば完了。

In [31]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)


---

## 実践

実際にモデルを作って文章を生成してみよう。さっきのモデルを使う。

In [32]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 8,725,312


パラメータ数は一緒。

### 学習

学習させる。

In [33]:
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model, trunc_len=100):
    model.eval()
    losses = []
    for x, t in test_loader:
        h = None
        for i in range(0, x.shape[1], trunc_len):
            x_batch = x[:, i:i+trunc_len].to(device)
            t_batch = t[:, i:i+trunc_len].to(device)
            y, h = model(x_batch, h)
            loss = loss_fn(y, t_batch)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl

def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: Truncated BPTTで区切る長さ

    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            h = None

            # Truncated BPTT
            for i in range(0, x.shape[1], trunc_len):
                trunc_x = x[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len, n_vocab)
                trunc_t = t[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len)

                optimizer.zero_grad()
                y, h = model(trunc_x, h)
                loss = loss_fn(y, trunc_t)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 計算グラフから切り離す
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [34]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [35]:
train(model, optimizer, trunc_len=100, n_epochs=20, prog_unit=2)

  1-2/20: #################### 100% [00:05:37.56] ppl train: 592.01, test: 428.68 
  3-4/20: #################### 100% [00:05:34.62] ppl train: 244.16, test: 295.80 
  5-6/20: #################### 100% [00:05:34.45] ppl train: 160.15, test: 239.83 
  7-8/20: #################### 100% [00:05:39.80] ppl train: 121.69, test: 218.85 
 9-10/20: #################### 100% [00:05:35.66] ppl train: 98.81, test: 201.81 
11-12/20: #################### 100% [00:05:28.61] ppl train: 85.83, test: 192.93 
13-14/20: #################### 100% [00:05:36.58] ppl train: 72.47, test: 188.22 
15-16/20: #################### 100% [00:05:37.88] ppl train: 63.89, test: 187.10 
17-18/20: #################### 100% [00:05:41.25] ppl train: 56.50, test: 185.86 
19-20/20: #################### 100% [00:05:40.31] ppl train: 51.19, test: 186.05 


In [36]:
model_path = "models/lm_rnn.pth"
torch.save(model.state_dict(), model_path)

訓練データのpplは順調に下がっている一方でテストデータの方は途中で止まってしまった。過学習ということだね。まあとりあえず学習は出来た。

### 文章生成

学習したモデルを使って文章を生成する。モデルが出力した単語を次の入力とし、再帰的に出力を得る。

In [37]:
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [38]:
def token_sampling(y):
    y = y.squeeze()
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = "",
    max_len: int = 100
) -> str:
    model.eval()

    # 隠れ状態の初期値
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device).unsqueeze(0)
    y, h = model(x)
    next_token = token_sampling(y)
    token_ids.append(next_token)

    # 続きの文章生成
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).unsqueeze(0)
        y, h = model(x, h)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [39]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

本unc053mとなっており、複数のグルロが1879年に拾えるカンパニー・スペラーマンを設立し、佐多稲中海線においては、本系列にクリスタルノが降り、当時はドイツ提辞任では初めて、経済学の発症が終わってしまった。享年107、より分ごっこの間に助けられずに国内が現れたため、急降下の位置を計測人にするなど前向きとなる。2018年
使わない。 冷は、1956年出張晩年は未満を横うため、ほか結成・ド・ラ・ヴィーヌ」の艦隊とウォーソ自分の内閣を置いて大きな被害を受けて、汲桑に残る突撃隊を護衛のため帝国寄りの住民が殺害された。1948年8月で、「盾は楚父・サンクスゾテカのような事を知る。2月までは「鹿理の状態で深ドイツを結ぶ男、フランス
2002年(平成11年)4月8日に全日本女子サッカー選手権大会から2000年にオリジナル連盟賞 2015年11月、ロシア政府文化センター展開 (JSAy)、葉戦隊大使館機15090mm、21-871 ホルガ地区を実現しみ、モロン政策賞のトップの可能性を優先することを発表し、同デモの航力に貢献していた。しかし理由がポスターに協議しており、特にフランス
明治プロダクションは続けられた。塩族民兵空港は、従来のバスでは検出自称・フィルズの仕事への功際に料をしており、対象上した第1作では190～0.4.54mmの値上げに変更していたが、解体指導により発表されたなど駅を発表したが異なり、頼り咲年で1O+〜1987年3月 [査読有り]RIRO 2015時のキャッシュアーティクスの
彼女の作品は概恋の42地を辞と考える形で逃げ助したうえで、これはマルクによる出産物資性についてのいいも、展少女の中常に机も再生くが、アニメ化粧品として169個名のみ)があり、顧客用地、50身.47妻として収録されている。また、ホーン・エアゴ・ピストハウスはマルヴァン・始アレクサンダーショップ・コリーズ、ブ


マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減ったと思われる。

### モデルの改良

少しだけモデルをいじって改良する。

- PyTorchのRNN層を用いる

    私が先で実装したRNN層は効率的でないようで、PyTorchのRNN層を用いた方が学習が早かった。

- ドロップアウト層を追加する

    RNNでは過学習を抑える手法としてドロップアウト層の導入が良く使われる。

In [40]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        x = self.dropout(x)
        hs, h = self.rnn(x, h) # (batch_size, seq_length, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, h

In [43]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

optimizer = optim.Adam(model.parameters(), lr=1e-4)

num of parameters: 8,725,312


ドロップアウト層を追加しただけなのでパラメータ数に変動はない。

学習させてみよう。なんとなく学習回数を増やしておく。

In [44]:
train(model, optimizer, trunc_len=100, n_epochs=40, prog_unit=2)

  1-2/40: #################### 100% [00:03:33.61] ppl train: 669.66, test: 476.40 
  3-4/40: #################### 100% [00:03:34.96] ppl train: 317.59, test: 338.21 
  5-6/40: #################### 100% [00:03:40.61] ppl train: 225.40, test: 273.76 
  7-8/40: #################### 100% [00:03:36.96] ppl train: 181.45, test: 249.26 
 9-10/40: #################### 100% [00:03:35.14] ppl train: 154.11, test: 221.80 
11-12/40: #################### 100% [00:03:32.63] ppl train: 138.90, test: 217.23 
13-14/40: #################### 100% [00:03:34.66] ppl train: 124.82, test: 205.19 
15-16/40: #################### 100% [00:03:39.87] ppl train: 115.47, test: 195.41 
17-18/40: #################### 100% [00:03:39.57] ppl train: 107.66, test: 188.58 
19-20/40: #################### 100% [00:03:43.58] ppl train: 101.78, test: 187.58 
21-22/40: #################### 100% [00:03:32.45] ppl train: 97.18, test: 181.23 
23-24/40: #################### 100% [00:03:25.94] ppl train: 91.58, test: 178.86 
25-26/

In [45]:
model_path = "models/lm_rnn_imp.pth"
torch.save(model.state_dict(), model_path)

先程よりも学習速度が上がった。そして訓練データとテストデータでpplの差が小さくなった。過学習が抑えられたと言える。

文章を生成してみよう。

In [46]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

旧譜類の高野食一が少佐、八法会議が早いわらか、次大戦後・学的虐待の学校の配下にアメリカ軍ことが判明した。
政が必要な名やコンゴリーでの監視に発生することなどがいていた。
ボドルタ・29メルスタは、自身の歌すこと盤商品が1938年振り10曲オペラの登山口を馬度を果たした。
香川口沼与によると本願寺派の有力の開設が翌年ハン末2川で、引退後は石勒が下京してその旨を図ったところ、1年逝去のめもつれは催された。この影響でもゲーリングは「音楽今が、それはなじて残すところがっただ、も広大な行方が肉にあまごうという。
しかし、アメリカはリヤード・ニュージーラ・壱と(Bヴィス)復活のために資金としてマスジャースとはいを使用しなかった。以降、1811が目に赴いた(He Gren Fame granunder to nues Sergyoren Stara, and gve band であること 9 25e 48 Maxot to the O LIG


先程よりテストデータでのpplが小さいモデルを使っているので、少しは質の高い文章になっているはずだが、まあこの程度ではほとんど変わらないかな。